<h1 align="center">Programación Científica en Python</h1>
<h2 align="center">Descripción de Proyectos</h2> 
<h2 align="center">Por: Maximiliano Bombin Sanhueza</h2> 
<h2 align="center">Rol: 201104308-1</h2> 


Para los proyectos descritos a continuación, los tres componentes siguientes serán evaluados con igual ponderación:

1. __Implementación__: Código generado para la implementación, usando _algunas_ de las herramientas aprendidas en este curso.
2. __Profiling__: Mediciones de la _performance_ de su solución al problema.
3. __Visualización__: Visualización de los resultados utilizando `Matplotlib`.

En cualquier caso usted es libre de diseñar los _experimentos_, _mediciones de performance_ y de los _resultados y gráficos_ a mostrar.

## 1.- Cellular Automaton: Conway's Game of Life

El _Juego de la Vida_ es una aplicación de autómatas celulares (conjunto de reglas), para simular la formación de patrones en el crecimiento de colonias de organismos biológicos.


Este juego se representa por medio de un arreglo bi-dimensional de __células vivas__ y __células muertas__. Las reglas para pasar de una generación a la otras son las siguientes (_Existen diferentes variaciones, pero estas son las más comunes_):

* __Sobrepoblación__: Si una célula viva es rodeada por más de tres células vivas, muere.
* __Estasis__: Si una célula viva es rodeada por dos o tres células vivas, sobrevive.
* __Subpoblación__: Si una célula viva es rodeada por menos de dos células vivas, muere.
* __Reproduction__: Si una célula muerta es rodeada por exáctamente tres células vivas, esta se vuelve una célula viva.

Aquí cada célula es representada como un píxel en una grilla/arreglo bi-dimensional.

Para más información visitar los siguientes links:
* [https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life)
* [https://jakevdp.github.io/blog/2013/08/07/conways-game-of-life/](https://jakevdp.github.io/blog/2013/08/07/conways-game-of-life/)
* [https://bitstorm.org/gameoflife/](https://bitstorm.org/gameoflife/)

## 2.- Dynamic Systems: N-body Simulation

Este consiste en la simulación numérica de un sistem de N EDOs (Ecuaciones Diferenciales Ordinarias), que representan la dinámica de movimiento de un sistema de N partículas interactuando entre sí, como por ejemplo, en la interacción gravitacional de planetas y estrellas.

Sean un conjunto de $N$ cuerpos con masas $\{m_1, m_2, \ldots, m_N \}$, posiciones (2D) $\{\mathbf{x_1}, \mathbf{x_2}, \ldots, \mathbf{x_N} \}$ y velocidades $\{\mathbf{v_1}, \mathbf{v_2}, \ldots, \mathbf{v_N}\}$, que interactúan gravitacionalmente de acuerdo a la __Ley de Gravitación Universal__:
$$
    \mathbf{F_{ij}} = G \frac{m_i m_j}{|| \mathbf{x_j} - \mathbf{x_i} ||^3}(\mathbf{x_j} - \mathbf{x_i}),
$$
correspondiente a la fuerza ejercida __sobre__ el cuerpo `i`, __por__ el cuerpo `j`. Entonces la dinámica del sistema viene descrita por la siguiente ecuación de movimiento (2da Ley de Newton):
$$
    m_i \frac{d}{dt} \mathbf{v_i} = m_i \frac{d^2}{dt^2}\mathbf{x_i} = \sum_{j\neq i}^{N} \mathbf{F_{ij}} = \sum_{j \neq i}^{N} G \frac{m_i m_j}{|| \mathbf{x_j} - \mathbf{x_i} ||^3}(\mathbf{x_j} - \mathbf{x_i})
$$

Para el caso 2D: $\mathbf{x_i} = (x_i, y_j)$, $\mathbf{v_i} = (v_{x_i}, v_{y_i})$ y las ecuaciones que describen el movimiento quedan:
$$
  \frac{d^2}{dt^2}x_i = \frac{d}{dt}v_{x_i} = G \ \sum_{j\neq i}^{N}\frac{m_j}{(x_j-x_i)^2} = H(x_i)
$$

$$  
  \frac{d^2}{dt^2}y_i = \frac{d}{dt}v_{y_i} = G \ \sum_{j\neq i}^{N}\frac{m_j}{(y_j-y_i)^2} = H(y_i)
$$

Se requiere resolver __numéricamente__ la EDO para cada partícula del sistema. Para ello en cada componente de $\mathbf{x_i}$ y $\mathbf{v_i}$, se utiliza el método de Euler del siguiente modo ($\Delta t$ _time step_ y $k$ número de iteración):
$$
\Rightarrow v_{x_i}^{(k+1)} = H(x_i^{(k)}) \Delta t + v_{x_i}^{(k)} 
$$
$$
x_i^{(k+1)} = v_{x_i}^{(k+1)} \Delta t + x_i^{(k)} 
$$


$$
\Rightarrow v_{y_i}^{(k+1)} = H(y_i^{(k)}) \Delta t + v_{y_i}^{(k)}
$$
$$
y_i^{(k+1)} = v_{y_i}^{(k+1)} \Delta t + y_i^{(k)}
$$
  
(Primero se actualiza la velocidad, luego la posición). Estas dos iteraciones deben llevarse a cabo para todas las partículas del sistema, partiendo de condiciones iniciales $(x_i^0, y_i^0)$ y $(v_{x_i}^0, v_{y_i}^0)$ para todas las partículas.

In [1]:
#FUNCIONES

%load_ext line_profiler

import numpy as np
import matplotlib.pyplot as plt
import numba
import base64

from ipywidgets import widgets


#Función "H"
def H(componente_posicion, componente_posiciones, masas):
    G = 6.67300e-11
    suma = 0
    
    for i in range(len(componente_posiciones)):
        
        if (componente_posiciones[i] != componente_posicion):
            suma += masas[i]/((componente_posiciones[i] - componente_posicion)**2)
    
    return G*suma



#Euler
def euler(H, x_posiciones, y_posiciones, x_velocidades, y_velocidades, masas, dt, k):
    historia_total = []
    
    for i in range(len(x_posiciones)):
        historia_de_posiciones_particula= []
        
        pos_x = x_posiciones[i]
        pos_y = y_posiciones[i]
        
        v_x = x_velocidades[i]
        v_y = y_velocidades[i]
        
        historia_de_posiciones_particula.append([pos_x,pos_y])
        
        for j in range(k):
            v_x = v_x + dt * H(pos_x, x_posiciones, masas)
            v_y = v_y + dt * H(pos_y, y_posiciones, masas)
            
            pos_x = pos_x + dt * v_x
            pos_y = pos_y + dt * v_y
            
            historia_de_posiciones_particula.append([pos_x,pos_y])
        
        historia_total.append(historia_de_posiciones_particula)
    
    return np.array(historia_total)


#Funcion de Graficado
def graficar(historia_del_sistema, tf):
    plt.figure(figsize=(7,6))
    plt.xlim(int(np.amin(historia_del_sistema[:,:,0]))-1,int(np.amax(historia_del_sistema[:,:,0]))+1)
    plt.ylim(int(np.amin(historia_del_sistema[:,:,1]))-1,int(np.amax(historia_del_sistema[:,:,1]))+1)
    plt.xlabel('X')
    plt.ylabel('Y')
    color = np.arange(len(historia_del_sistema[:,tf,0]))
    plt.title('Evolución del Sistema')
    plt.scatter(historia_del_sistema[:,tf,0], historia_del_sistema[:,tf,1], linewidth=2, c = color)
    plt.grid()
    
    plt.show()



In [39]:
#PARTICULAS 

N = 100

#Posicion
x_posiciones = np.random.random(N)
y_posiciones = np.random.random(N)

#velocidad
x_velocidades = np.random.random(N)
y_velocidades = np.random.random(N)

#masa

masas = np.random.random(N)

#constantes de tiempo e iteraciones

dt = 0.05
k = 400

#PROGRAMA

historia_del_sistema = euler(H, x_posiciones, y_posiciones, x_velocidades, y_velocidades, masas, dt, k)




In [40]:
dp = 1
widgets.interact(graficar,historia_del_sistema=widgets.fixed(historia_del_sistema),tf=(0,k,dp))

C:\Users\Max\Anaconda3\lib\site-packages\traitlets\traitlets.py:567: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  silent = bool(old_value == new_value)


<function __main__.graficar>

In [4]:
#2 ANÁLISIS DEL TIEMPO

t1 = %timeit -o -n 10 euler(H, x_posiciones, y_posiciones, x_velocidades, y_velocidades, masas,dt,k)

10 loops, best of 3: 6.02 s per loop


In [5]:
print("\nTiempo de Compilación:")
print(t1.compile_time)

print("\nPeor Tiempo:")
print(t1.worst)

print("\nMejor Tiempo:")
print(t1.best)





Tiempo de Compilación:
7.389219391765021e-05

Peor Tiempo:
6.3639593306710704

Mejor Tiempo:
6.015589821802987


In [6]:
#Analizamos El programa.

%prun -s tottime -q -l 15 -T prun0 euler(H, x_posiciones, y_posiciones, x_velocidades, y_velocidades, masas, dt, k)


 
*** Profile printout saved to text file 'prun0'. 


In [7]:
print(open('prun0', 'r').read())

         200208 function calls in 5.959 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    80000    5.865    0.000    5.869    0.000 <ipython-input-1-2c3d60891266>:14(H)
        1    0.077    0.077    5.958    5.958 <ipython-input-1-2c3d60891266>:28(euler)
        1    0.009    0.009    0.009    0.009 {built-in method numpy.core.multiarray.array}
    80001    0.004    0.000    0.004    0.000 {built-in method builtins.len}
    40200    0.003    0.000    0.003    0.000 {method 'append' of 'list' objects}
        1    0.001    0.001    5.959    5.959 <string>:1(<module>)
        1    0.000    0.000    5.959    5.959 {built-in method builtins.exec}
        2    0.000    0.000    0.000    0.000 cycler.py:227(<genexpr>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}


In [8]:
#Se puede apreciar del listado anterior que el mayor tiempo acumulado está presente en la función "euler",
#pero esta función corresponde a nuestro programa completo, y podemos ver que la función que ocupa la mayor parte del tiempo
#asociado a la función "euler" es la función "H" (por lejos). Para Optimizar el códido debemos analizar y arreglar ésta función.

#Haremos un line profiling.

%lprun -T lprof -f H H(x_posiciones[0], x_posiciones, masas)


*** Profile printout saved to text file 'lprof'. 


In [9]:
print(open('lprof', 'r').read())

Timer unit: 2.92064e-07 s

Total time: 0.000216711 s
File: <ipython-input-1-2c3d60891266>
Function: H at line 14

Line #      Hits         Time  Per Hit   % Time  Line Contents
    14                                           def H(componente_posicion, componente_posiciones, masas):
    15         1            5      5.0      0.7      G = 6.67300e-11
    16         1            3      3.0      0.4      suma = 0
    17                                               
    18       101          140      1.4     18.9      for i in range(len(componente_posiciones)):
    19                                                   
    20       100          181      1.8     24.4          if (componente_posiciones[i] != componente_posicion):
    21        99          409      4.1     55.1              suma += masas[i]/((componente_posiciones[i] - componente_posicion)**2)
    22                                               
    23         1            4      4.0      0.5      return G*suma


In [24]:
#Podemos ver que más del 50% del tiempo está asociado a la línea 30, lo cual tiene sentido ya que corresponde al cálculo
#más complejo de toda la función. Otro factor que también inluye es el cálculo constante del porte del arreglo en la línea 27
#por medio de la función len, que al incrementar las iteraciones empieza claramente a acumularse y de manera bastante
#innecesaria. También Podemos Notar que la línea del "if" que busca que se excluya al elemento especificado en la suma
#al aumentar las iteraciones empieza a tomar bastante tiempo, considerando que esta pregunta solo es válida para cuando
#aún no nos hemos topado con tal número, una vez que ya se encontró tal número y se excluyó de la suma no es necesario
#seguir preguntado.

#Intentaremos resolver estos problemas utilizando las optimizaciones de "numba" y calculando el largo del arreglo una sola vez,
#guardando el resultado en una sola variable.

@numba.jit('float64 (float64,float64[:],float64[:])', nopython=True)
def H2(componente_posicion, componente_posiciones, masas):
    G = 6.67300e-11
    suma = 0
    total_posiciones = len(componente_posiciones)
    
    for i in range(total_posiciones):
        if(componente_posiciones[i] == componente_posicion):
            ultimo_indice = i + 1
            break
        suma += masas[i]/((componente_posiciones[i] - componente_posicion)**2)
    
    
    for j in range(ultimo_indice, total_posiciones):
        suma += masas[j]/((componente_posiciones[j] - componente_posicion)**2)
    
    return G*suma

In [25]:
#Resultados

%lprun -T lprof2 -f H2 H2(x_posiciones[0], x_posiciones, masas)

%lprun -T lprof3 -f H2 H2(x_posiciones[50], x_posiciones, masas)

%prun -s tottime -q -l 15 -T prun1 euler(H2, x_posiciones, y_posiciones, x_velocidades, y_velocidades, masas, dt, k)


*** Profile printout saved to text file 'lprof2'. 

*** Profile printout saved to text file 'lprof3'. 
 
*** Profile printout saved to text file 'prun1'. 


In [12]:
print(open('lprof2', 'r').read())

print ("")
print ("")
print(open('lprof3', 'r').read())

Timer unit: 2.92064e-07 s

Total time: 0 s
File: <ipython-input-10-471748d07aa1>
Function: H at line 12

Line #      Hits         Time  Per Hit   % Time  Line Contents
    12                                           @numba.jit('float64 (float64,float64[:],float64[:])', nopython=True)
    13                                           def H(componente_posicion, componente_posiciones, masas):
    14                                               G = 6.67300e-11
    15                                               suma = 0
    16                                               total_posiciones = len(componente_posiciones)
    17                                               
    18                                               for i in range(total_posiciones):
    19                                                   if(componente_posiciones[i] == componente_posicion):
    20                                                       ultimo_indice = i + 1
    21                                     

In [13]:
print(open('prun1', 'r').read())

         120218 function calls in 0.261 seconds

   Ordered by: internal time
   List reduced from 17 to 15 due to restriction <15>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.168    0.168    0.260    0.260 <ipython-input-1-2c3d60891266>:28(euler)
    80000    0.080    0.000    0.080    0.000 <ipython-input-10-471748d07aa1>:12(H)
        1    0.009    0.009    0.009    0.009 {built-in method numpy.core.multiarray.array}
    40200    0.002    0.000    0.002    0.000 {method 'append' of 'list' objects}
        1    0.001    0.001    0.261    0.261 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 module.py:66(_dispose)
        1    0.000    0.000    0.261    0.261 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 llvmthreadsafe.py:45(_ts_dispose)
        1    0.000    0.000    0.000    0.000 llvmthreadsafe.py:20(__enter__)
        2    0.000    0.000    0.000    0.000 ffi.py:119(close)
        1    0

In [43]:
#Podemos ver que el tiempo disminuyó considerablemente.
#Calculamos la razón de rendimiento obtenida
t2 = %timeit -o -n 10 euler(H2, x_posiciones, y_posiciones, x_velocidades, y_velocidades, masas, dt, k)


10 loops, best of 3: 132 ms per loop


In [44]:
print("Mejora obtenida:")
print(t1.best/t2.best)

Mejora obtenida:
45.71228088007854


In [41]:
historia_del_sistema2 = euler(H2, x_posiciones, y_posiciones, x_velocidades, y_velocidades, masas, dt, k)

In [42]:
print (historia_del_sistema== historia_del_sistema2)

[[[ True  True]
  [ True  True]
  [ True  True]
  ..., 
  [ True  True]
  [ True  True]
  [ True  True]]

 [[ True  True]
  [ True  True]
  [ True  True]
  ..., 
  [ True  True]
  [ True  True]
  [ True  True]]

 [[ True  True]
  [ True  True]
  [ True  True]
  ..., 
  [ True  True]
  [ True  True]
  [ True  True]]

 ..., 
 [[ True  True]
  [ True  True]
  [ True  True]
  ..., 
  [ True  True]
  [ True  True]
  [ True  True]]

 [[ True  True]
  [ True  True]
  [ True  True]
  ..., 
  [ True  True]
  [ True  True]
  [ True  True]]

 [[ True  True]
  [ True  True]
  [ True  True]
  ..., 
  [ True  True]
  [ True  True]
  [ True  True]]]
